In [1]:
#!pip install datasets transformers

In [2]:
#import torch

In [3]:
import pickle
import re
import pandas as pd
import numpy as np
from datasets import Dataset

from transformers import BertTokenizer,BertForSequenceClassification,TrainingArguments,Trainer

C:\Users\tungb\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\tungb\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\tungb\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


Tiền xử lý dữ liệu

In [4]:
def preprocessing_data(text, tokenizer, max_word_length):
    df=pd.DataFrame.from_dict({'text' : [text]})

    df['text']=df['text'].apply(lambda x: x.lower())
    df['text']=df['text'].apply(lambda x: x.replace("#", ""))
    df['text']=df['text'].apply(lambda x: re.sub(r'@\w+\s*', '', x))
    df['text']=df['text'].apply(lambda x: re.sub(r'http\S+|www\S+', '', x))
    df['text']=df['text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
    df['text']=df['text'].apply(lambda x: re.sub(r'\d+', '', x))
    df['text']=df['text'].apply(lambda x : re.compile(r'<.*?>').sub(r'', x))

    df['input_ids']=df['text'].apply(lambda x: tokenizer(x, max_length=max_word_length, padding='max_length')['input_ids'])
    df=df[['input_ids']]

    return Dataset.from_pandas(df)

BERT Model

In [5]:
with open('bert_model.pkl', 'rb') as f:
    data = pickle.load(f)

# Load the pre-trained model from the .pkl file
#data = torch.load('bert_model (1).pkl', map_location=torch.device('cpu'))


tokenizer = data['tokenizer']
max_word_length = data['max_word_length']
model=data['model']
args=data['args']

Input đầu vào:

In [6]:
#text="Wrecked today got my hattrick ????"
#text="Just happened a terrible car crash"
#text="http://t.co/OK9EBHurfl If your neighborhood school falls into a sinkhole State Sen. Steve Yarbrough may be close by..."
text='Welcome home Dragon! \n@SpaceX\n’s Dragon cargo spacecraft splashed down off the coast of Florida today at 10:30am ET carrying 3,600 pounds of science and hardware back to Earth. This marks the completion of SpaceX’s 28th cargo resupply mission.'

In [7]:
output_ds=preprocessing_data(text,tokenizer,max_word_length)

In [8]:
output_ds

Dataset({
    features: ['input_ids'],
    num_rows: 1
})

In [9]:
trainer = Trainer(
    model=model,
    args=args, 
)

In [10]:
outputs=trainer.predict(output_ds)

Kết quả dự đoán:

In [11]:
outputs.predictions.argmax(1)

array([1], dtype=int64)